In [114]:
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from collections import defaultdict 
from imdb import IMDb
import time
import requests

### Scrape the movie gross Information from boxofficemojo.com 

In [99]:
years = xrange(2009,2015)
pages = xrange(1,9)
year_pagetxt = {}
for year in years: 
    pagestext = {}
    for page in pages: 
        r = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=%s&view=releasedate&view2=domestic&yr=%s&p=.htm"%(page, year))
        pagestext[page] = r.text
        time.sleep(1)
    year_pagetxt[year] = pagestext

In [103]:
movie_budget = defaultdict(list) 
for year in years: 
    for page in pages: 
        soup = BeautifulSoup(year_pagetxt[year][page], "html.parser")
        rows = soup.find_all("font", attrs={'size':'2'})
           
        start = 10 
        for i in range(start,len(rows)-2):
            t = rows[i].get_text()
            if unicode('Summary of') in t: 
                break
            elif (i-start) % 9 == 0: 
                movie_budget['rank'].append(t)
            elif (i-start) % 9 == 1: 
                movie_budget['year'].append(year)
                r = '('+str(year)+')'
                if unicode(r) in t: 
                    j = t.index(unicode(r))
                    movie_budget['title'].append(t[:j])
                else: 
                    movie_budget['title'].append(t)
            elif (i-start) % 9 == 2: 
                movie_budget['studio'].append(t)
            elif (i-start) % 9 == 3: 
                movie_budget['gross'].append(t)
            elif (i-start) % 9 == 4: 
                movie_budget['gross theaters'].append(t)
            elif (i-start) % 9 == 5: 
                movie_budget['opening'].append(t)
            elif (i-start) % 9 == 6: 
                movie_budget['opening theaters'].append(t)
            elif (i-start) % 9 == 7: 
                movie_budget['open'].append(t)
            elif (i-start) % 9 == 8: 
                movie_budget['close'].append(t)

###Scrape movie budget information from the-numbers.com 

In [59]:
r_numbers = requests.get("http://www.the-numbers.com/movie/budgets/all")

d_=pq(r_numbers.text)
d_tables=pq(d_('table'))
rows = pq(d_tables[0])('tr')

In [60]:
budget = defaultdict(list)
for j in range(1,len(rows)):
    dat = pq(rows[j])('td')
    for i in range(len(dat)): 
        if i % 6 == 1:
            budget['year'].append(int(pq(dat[i])('a').text().split('/')[-1]))
        elif i % 6 == 2:
            budget['title'].append(pq(dat[i])('a').text())
        elif i % 6 == 3: 
            budget['budget'].append(pq(dat[i]).text())

### Search_movies might return many movies related to our search. This finds the appropriate movie.

In [121]:
def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            continue
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i] == sorted_mtitle[i]:
                        counts[j] += 1
            else:
                continue
        k = counts.index(max(counts))
        if len(year_list) >= 1:
            return year_list[k]
        else:
            return None

### Combine movie ids, budget info, and gross info into one dict

In [ ]:
ia = IMDb(accessSystem='http')

#The next process actually attempts to match the movies in the Box Office dataframe to the 
BOmissingmovies = list() # To hold anything we don't find
BOdict = {} # This will actually contain the movies and information about them.

movienumber = len(movie_budget['gross'])

for i in range(0,movienumber):
    movieobj = None
    # Need to process the row-level information out of BOdf in order to get the movie objects
    movieobj = ia.search_movie(movie_budget['title'][i])
    
    #Handling cases where we haven't found the movie or have multiples
    if movieobj is None or len(movieobj)>1:
        movieobj = find_movie(movie_budget['title'][i], movie_budget['year'][i], ia.search_movie(movie_budget['title'][i])) # find the movie

    if movieobj is not None and not (type(movieobj) == list and len(movieobj) == 0):
        ## Get movie id ##
        if type(movieobj) == list: 
            movieobj = movieobj[0]
        ia.update(movieobj)    
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        BOdict[movid] = {}
        # "title": title of movie
        BOdict[movid]['title'] = movie_budget['title'][i]
        BOdict[movid]['gross'] = movie_budget['gross'][i]
        BOdict[movid]['opening'] = movie_budget['opening'][i]
        BOdict[movid]['rank'] = movie_budget['rank'][i]
        BOdict[movid]['studio'] = movie_budget['studio'][i] 
        BOdict[movid]['open'] = movie_budget['open'][i]
        BOdict[movid]['close'] = movie_budget['close'][i]
        BOdict[movid]['opening theaters'] = movie_budget['opening theaters'][i]
        try:
            BOdict[movid]['rating'] = movieobj['rating']
        except: 
            BOdict[movid]['rating'] = []
        try:
            BOdict[movid]['mpaa'] = movieobj['mpaa']
        except: 
            BOdict[movid]['mpaa'] = []
        try:
            BOdict[movid]['director'] = movieobj['director']
        except: 
            BOdict[movid]['director'] = []
        try:
            BOdict[movid]['cast'] = movieobj['cast']
        except:
            BOdict[movid]['cast'] = []
        #BOdict[movid]['']
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
    else:
        BOmissingmovies.append((i, movie_budget['title'][i], movie_budget['year'][i]))
        